### Get stock prices data from polygon API

In [1]:
#!pip install -r requirements.txt

In [2]:
from stockify import * 
import datetime as dt 
from polygon import RESTClient
from polygon.rest import models
from http import HTTPStatus
    
API_key = 'PuJHla6_U_GDHcNE3XSpk7L2gCCpdJ9J'
client = RESTClient(API_key)

In [3]:
## Put the API key (if we do pay) into ENV_VARS
current_date = dt.datetime.now().date()
stocks_tickers = get_list_stocks_tickers(text_file_name='stocks_list',
                                         only_stock_tickers=True)

stocks_requirments = get_list_stocks_tickers(text_file_name='stocks_list',
                                         only_stock_tickers=False)

prices_dfs = []
dividends_dfs = []
DP_ratio_dfs = []

stocks_tickers=['SCM','ARR','AGNC']
for st in stocks_tickers:
    
    ##Prices process
    #try:
        p_raw_response = get_p_process_by_ticker(API_key,
                                            client,
                                            st,
                                            get_daily_time_buckets(max_lookback_years=2,
                                                                    current_is_timestamp=False,
                                                                    frequency='daily',
                                                                    include_weekend=False),
                                            stock_prices_frequency= 'daily')
        
        pdf = get_jsonList_prices_into_dataframe(get_serialized_price_raw_into_json(p_raw_response,st),
                                    ['close'])
    # If exceed request per minute
    #except (BadResponse) as e:
    #    continue
    #    print(e)

        prices_dfs.append(pdf)
    
    ##Dividends process
    
        d_raw_response = get_top_x_dividends_by_ticker(ticker=st,
                                                    client=client,
                                                    API_latency_secs=1.5,
                                                    n_API_calls=24
                                                    )
        

        ddf = get_jsonList_dividends_into_dataframe(DivjsonList = d_raw_response,
                                                    measurable_time_variables= ['pay_date'],
                                                    dividend_types=['CD'])
                      
        ##SC = special dividens; all the others paid out are ordinary are marked as CD by the API.
        
        dividends_dfs.append(ddf)
        
        DP_ratio_df = get_dividend_price_ratios(ticker=st,
                                                prices_df=pdf,
                                                dividends_df=ddf)
        
        DP_ratio_dfs.append(DP_ratio_df)


if len(prices_dfs)>0: 
    prices_dfs = pd.concat(prices_dfs)
    
if len(dividends_dfs)>0: 
    dividends_dfs = pd.concat(dividends_dfs)
    
if len(DP_ratio_dfs)>0: 
    DP_ratio_dfs = pd.concat(DP_ratio_dfs)

In [4]:
DP_ratio_dfs[DP_ratio_dfs['converted_utc_timestamp']=='2022-12-28 05:00:00']

,stock,close,converted_utc_timestamp,n_paid_dividens_current_year,sum_paid_dividens_current_year,DP_ratio
0,SCM,13.07,2022-12-28 05:00:00,12.0,1.1196,8.566182
0,ARR,5.59,2022-12-28 05:00:00,12.0,1.2000,21.466905
0,AGNC,10.41,2022-12-28 05:00:00,12.0,1.4400,13.832853
